In [24]:
import pandas as pd
import numpy as np
from keras import models, Sequential
from keras.layers import BatchNormalization, Conv2D, Activation, Dropout, GlobalAveragePooling2D, add, Input, Dense,MaxPool2D
from keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import accuracy_score,confusion_matrix
from keras.utils.np_utils import to_categorical
from sklearn.model_selection import StratifiedKFold
from keras.utils import np_utils
from keras import regularizers
from keras import optimizers
from keras.callbacks import EarlyStopping
import sys
sys.path.append('/root/oneclick/CFSC/CFSC/')
from Training.Networks import Res50




In [25]:
def bn_relu(layer, dropout=0, **params):
    layer = BatchNormalization()(layer)
    layer = Activation(params['conv_activation'])(layer)

    if dropout > 0:
        layer = Dropout(dropout)(layer)
    return layer


def global_average_pooling(layer, cls):
    layer = Conv2D(cls, [1, 1])(layer)
    layer = GlobalAveragePooling2D()(layer)
    layer = Activation(activation='softmax')(layer)
    return layer

In [26]:
def resnet_block_A(layer, filters, kernels, dropout, activation, cross_block=False,shrink = False):
    # -Conv-BN-Act-Conv-BN-Act-
    # ↳-----Conv-BN-Act-------↑

    shape = [1, 1]
    if shrink:
        shape = [2, 2]

    if cross_block:
        shortcut = Conv2D(filters=filters,
                          kernel_size=shape,
                          kernel_initializer='random_uniform',
                          # kernel_regularizer=regularizers.l2(0.01),
                          strides=shape,
                          padding='same')(layer)
        shortcut = bn_relu(shortcut,conv_activation='relu')
    else:
        shortcut = layer

    layer = Conv2D(filters=filters,
                   kernel_size=kernels,
                   kernel_initializer='random_uniform',
                   # kernel_regularizer=regularizers.l2(0.01),
                   strides=shape,
                   padding='same')(layer)
    layer = bn_relu(layer, dropout=dropout, conv_activation=activation)

    layer = Conv2D(filters=filters,
                   kernel_size=kernels,
                   kernel_initializer='random_uniform',
                   # kernel_regularizer=regularizers.l2(0.01),
                   strides=[1, 1],
                   padding='same')(layer)


    layer = bn_relu(layer, dropout=dropout, conv_activation=activation)

    layer = add([shortcut, layer])

    # if shrink:
    #     layer = MaxPool2D()(layer)

    return layer

In [27]:
def Resnet_A(num):
    input = Input(shape=[100, 100, 1])
    #100*100
    layer1 = resnet_block_A(layer = input, filters= 32, kernels=[3,3],
                           dropout= 0,activation = 'relu',cross_block= True)
    layer2 = resnet_block_A(layer = layer1, filters= 32, kernels=[3,3],
                           dropout= 0,activation = 'relu')
    #50*50
    layer3 = resnet_block_A(layer=layer2, filters=64, kernels=[3, 3],
                           dropout=0, activation='relu',cross_block= True,shrink=True)
    layer4 = resnet_block_A(layer = layer3, filters= 64, kernels=[3,3],
                           dropout= 0,activation = 'relu')
    #25*25
    layer5 = resnet_block_A(layer=layer4, filters=128, kernels=[3, 3],
                           dropout=0, activation='relu', cross_block=True, shrink=True)
    layer6 = resnet_block_A(layer=layer5, filters=128, kernels=[3, 3],
                                   dropout=0, activation='relu')
    #13*13
    layer7 = resnet_block_A(layer=layer6, filters=256, kernels=[3, 3],
                           dropout=0, activation='relu', cross_block=True, shrink=True)
    layer8 = resnet_block_A(layer=layer7, filters=256, kernels=[3, 3],
                                   dropout=0, activation='relu')
    #7*7
    layer9 = resnet_block_A(layer=layer8, filters=512, kernels=[3, 3],
                           dropout=0, activation='relu', cross_block=True, shrink=True)
    layer10 = resnet_block_A(layer=layer9, filters=512, kernels=[3, 3],
                                   dropout=0, activation='relu')


    output = global_average_pooling(layer10, num)
    model = Model(inputs = [input], outputs = [output])
    model.summary()
    return model

In [28]:
def train(model):
    num = 40000
    X_train = np.load('../data/train_x.npy')[0:num]
    Y_train = np.load('../data/train_y.npy')[0:num]
    #Y_train = Y_train[:]
    X_train = np.reshape(X_train, [num, 100, 100, 1])
    x_train, x_val, y_train, y_val = train_test_split(X_train, Y_train, test_size=0.1, random_state=42)
    y_train = pd.DataFrame(y_train)[0]
    y_val = pd.DataFrame(y_val)[0]
    # one-hot，5 category
    y_labels = list(y_train.value_counts().index)
    y_labels = list(range(157))
    # y_labels = np.unique(y_train)
    le = preprocessing.LabelEncoder()
    le.fit(y_labels)
    num_labels = len(y_labels)
    y_train = to_categorical(y_train.map(lambda x: le.transform([x])[0]), num_labels)
    y_val = to_categorical(y_val.map(lambda x: le.transform([x])[0]), num_labels)

    adam = optimizers.adam(lr=0.01)
    model.compile(loss='categorical_crossentropy',
                  optimizer=adam,
                  metrics=['accuracy'])
    monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=3, verbose=1, mode='auto')

    model.fit(x_train, y_train,
              batch_size=50,
              epochs=20,
              validation_data=(x_val, y_val),
              callbacks=[monitor])
    score = model.evaluate(x_val, y_val, verbose=0)
    val_loss = score[0]
    acc = score[1]
    model.save('../models/RCNN')
    X_test = np.load('../data/test_x.npy')
    Y_test = np.load('../data/test_y.npy')
    # X_train reshape to [40000,100,100]
    X_test = np.reshape(X_test, [4000, 100, 100, 1])
    Y_test = Y_test[:,0]
    score = model.predict(X_test)
    score = np.argmax(score, axis=1)
    score = accuracy_score(score, Y_test)

    print(score)


In [29]:
model = Resnet_A(157)
train(model)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            (None, 100, 100, 1)  0                                            
__________________________________________________________________________________________________
conv2d_155 (Conv2D)             (None, 100, 100, 32) 320         input_7[0][0]                    
__________________________________________________________________________________________________
batch_normalization_152 (BatchN (None, 100, 100, 32) 128         conv2d_155[0][0]                 
__________________________________________________________________________________________________
activation_145 (Activation)     (None, 100, 100, 32) 0           batch_normalization_152[0][0]    
__________________________________________________________________________________________________
conv2d_154

Train on 36000 samples, validate on 4000 samples
Epoch 1/20
36000/36000 [==============================] - 177s 5ms/step - loss: 3.2593 - acc: 0.2223 - val_loss: 3.0245 - val_acc: 0.2098
Epoch 2/20
36000/36000 [==============================] - 170s 5ms/step - loss: 2.3983 - acc: 0.4016 - val_loss: 2.0961 - val_acc: 0.4802
Epoch 3/20
36000/36000 [==============================] - 170s 5ms/step - loss: 1.5122 - acc: 0.6212 - val_loss: 2.0713 - val_acc: 0.5150
Epoch 4/20
36000/36000 [==============================] - 170s 5ms/step - loss: 1.1379 - acc: 0.7078 - val_loss: 1.2668 - val_acc: 0.6793
Epoch 5/20
36000/36000 [==============================] - 170s 5ms/step - loss: 0.9436 - acc: 0.7519 - val_loss: 1.1402 - val_acc: 0.7070
Epoch 6/20
36000/36000 [==============================] - 170s 5ms/step - loss: 0.8147 - acc: 0.7807 - val_loss: 1.1726 - val_acc: 0.6935
Epoch 7/20
36000/36000 [==============================] - 170s 5ms/step - loss: 0.7009 - acc: 0.8047 - val_loss: 1.0783 - v

IndexError: too many indices for array